In [ ]:
from nltk.book import *

# TTR

Při seznamování s Pythonem jsme mluvili o *type-token ratio* (poměr počtu unikátních typů v textu vůči celkovému počtu tokenů) jako o relativně jednoduchém způsobu, jak operacionalizovat lexikální bohatost.

In [ ]:
def ttr(text):
    """Type-token ratio (TTR) of ``text``.

    :param text: input text
    :return: float

    """
    return len(set(text)) / len(text)

Zmínili jsme také, že jeho nevýhodou je, že není nezávislý na délce textu. Některá gramatická slůvka nám nezbývá než používat stále dokola, takže delší texty jsou z hlediska naměřené bohatosti znevýhodněny. Intuitivně: v rámci jedné věty se nezřídka podaří, že každé slovo je jiné, a TTR je tedy rovno 1. V rámci odstavce už to bude spíš výjimka, v rámci celého textu velmi nepravděpodobné, tím spíš, čím delší ten text bude.

Z toho plyne, že výsledky naměřené přes TTR nemusí odpovídat intuitivní představě o lexikální bohatosti. Např. román *Moby Dick* se po přečtení pár stránek jeví jako všeobecně lexikálně bohatší než scénář k filmu *Monty Python and the Holy Grail*, ale TTR vyjde vyšší u toho druhého, protože je nápadně kratší:

In [ ]:
for text in [text1, text6]:
    print(f"{text.name}: {len(text)} tokenů, TTR {ttr(text):.2f}")

Jako rychlý způsob, jak provést smysluplnější porovnání, jsme zkusili výpočt TTR aplikovat na stejně dlouhé vzorky (výřezy) obou textů:

In [ ]:
for text in [text1, text6]:
    print(f"{text.name}: {len(text)} tokenů, TTR prvních 5000 slov {ttr(text[:5000]):.2f}")

Tady už čísla lépe odpovídají očekáváním, ale vkradla se nám nová pochybnost: nedošlo náhodou výřezem k jinému druhu zkreslení? Co když jsme náhodou vyřízli nápadně lexikálně chudou část jednoho textu, a naopak velmi bohatou část textu druhého? Jak provést porovnání založené na celých textech a zároveň odstínit vliv délky textu?

# Moving (average) TTR

Jedna z možností: spočítat hodnoty TTR v (překrývajících se) oknech stejné délky, která pokrývají celý text.

```
step = 100    # o kolik tokenů okno posouváme
window = 200  # šířka okna

tokens: 0  100 200 300 400 500
        |---|---|---|---|---|--->
        \______/    |   |
         window 1   |   |
            \______/    |
             window 2   |
                \______/
                 window 3
```

In [ ]:
def mttr(text, *, step=100, window=500):
    """Moving TTR of ``text``.

    :param text: input text
    :param step: by how many tokens the window advances each time
    :param window: width of window
    :return: list of floats (TTR values)

    """
    ttrs = []
    for start in range(0, len(text), step):
        sample = text[start:start+window]
        if len(sample) != window:
            break
        ttrs.append(ttr(sample))
    return ttrs

In [ ]:
mttr(text1)[:10]

In [ ]:
len(mttr(text1))

Z těchto hodnot lze následně spočítat průměr, tj. *moving average TTR* (MATTR).

In [ ]:
from statistics import mean

def mattr(text, **ttr_kwargs):
    """Moving average TTR of ``text``.

    :param ttr_kwargs: keyword arguments passed on to :func:`mttr`
    :return: float

    """
    ttrs = mttr(text, **ttr_kwargs)
    return mean(ttrs)  # nebo jednoduše sum(ttrs) / len(ttrs)

Hodnoty MATTR pro dva texty různé délky by měly být srovnatelné (při zachování stejných parametrů `step` a `window`), protože jsou výsledkem zprůměrování dílčích měření TTR na vzorcích stejné délky.

In [ ]:
for text in [text1, text6]:
    print(f"{text.name}: MATTR {mattr(text):.2f}")

# Grafy vývoje TTR pomocí Matplotlib

Sekvenci hodnot TTR můžeme vykreslit pomocí knihovny Matplotlib (viz https://matplotlib.org/cheatsheets/ -- vřele doporučuju, Matplotlib má mnoho funkcí, tohle v nich velmi usnadní orientaci -- a https://matplotlib.org/stable/tutorials/index).

In [ ]:
import matplotlib.pyplot as plt

ttrs = mttr(text1)
fig, ax = plt.subplots()
ax.plot(ttrs)

Když zadáme delší `step` a širší `window`, křivku "vyhladíme": získáme představu o dlouhodobějších trendech, naopak ztratíme přehled o lokálních fluktuacích.

In [ ]:
ttrs = mttr(text1, step=1000, window=5000)
fig, ax = plt.subplots()
ax.plot(ttrs)

V této podobě je ale vizualizace pořád trochu syrová, nepřehledná. Můžeme vylepšit různé aspekty. Osa x aktuálně odpovídá indexům jednotlivých oken, je jich tedy cca 250 v tomto případě, jak si můžeme snadno ověřit:

In [ ]:
len(ttrs)

Místo toho by bylo asi lepší, aby osa x odpovídala pozici v textu z hlediska počtu tokenů. Každou hodnotu TTR bychom mohli umístit např. na pozici odpovídající prostředku daného okna, pro které byla vypočtena.

Dále by bylo dobré v zájmu lepší čitelnosti graf rozprostřít do více řádků, a kvůli snazšímu porovnávání se zdrojovým textem naznačit hranice nějakých vyšších strukturních celků jako jsou kapitoly nebo scény. A pak samozřejmě formální záležitosti typu popisky os, celého grafu atp.

Funkce `plot_mttr_fancy` uvedená v následující buňce je v jistém smyslu maximalistická varianta. Není cílem pochopit vše do puntíku, spíš ukázat flexibilitu a šířku možností. Zkuste si ji projít a trochu si s ní pohrát -- některé části upravit nebo zakomentovat, porovnat si, jak se pak liší výstup. K čemu přesně jednotlivé metody slouží si můžete též dohledat v [dokumentaci API knihovny Matplotlib](https://matplotlib.org/stable/api/index). Pokud budete chtít experimentovat s vlastními vylepšeními, znovu jako zdroj inspirace stran toho, co je možné a jak to lze implementovat, doporučuju https://matplotlib.org/cheatsheets/.

In [ ]:
import math

def plot_mttr_fancy(text, *, text_name=None,
                    width=10, tokens_per_plot=7_000, ymin=None, ymax=None,
                    step=100, window=500):
    ttrs = mttr(text, step=step, window=window)
    minmax = [min(ttrs), max(ttrs)]
    ymin = max(0, math.floor(minmax[0] * 10) / 10) if ymin is None else ymin
    ymax = min(1, math.ceil(minmax[1] * 10) / 10) if ymax is None else ymax
    window_centers = [i*step + window/2 for i in range(len(ttrs))]
    xticks = []
    xlabels = []
    for i, (word, next_word) in enumerate(bigrams(text)):
        if word in {"CHAPTER", "SCENE"} and next_word.isdigit():
            xticks.append(i)
            xlabels.append(next_word)
    num_plots = math.ceil(len(text) / tokens_per_plot)
    fig, axs = plt.subplots(num_plots, 1, squeeze=False,
                            layout="constrained", figsize=(width, 3*num_plots))
    for i, ax in enumerate(axs.flat):
        ax.plot(window_centers, ttrs)
        for extreme in minmax:
            ax.axhline(extreme, color="gray", linestyle="dashed")
        ax.set_xticks(xticks)
        ax.set_xticklabels(xlabels)
        ax.set_xlabel("Chapters/Scenes")
        ax.set_ylabel(f"TTR in {window} token window")
        ax.set_xlim(i*tokens_per_plot, (i+1)*tokens_per_plot)
        ax.set_ylim(ymin, ymax)
        ax.grid()
    if text_name is None and hasattr(text, "name"):
        text_name = text.name
    if text_name is not None:
        fig.suptitle(f"TTR in {text_name}")
    # Objekty reprezentující graf by funkce měla vrátit, aby si je uživatel
    # případně mohl ještě dle libosti upravit.
    return fig, axs

In [ ]:
plot_mttr_fancy(text6);

In [ ]:
plot_mttr_fancy(text1[:50_000], text_name="first 50 000 tokens of " + text1.name);

In [ ]:
fig, axs = plot_mttr_fancy(text6[:7000], ymax=.7)
moby_ttrs = mttr(text1)
for ax in axs.flat:
    ax.axhline(min(moby_ttrs), color="C1", linestyle="dashed")
    ax.axhline(max(moby_ttrs), color="C1", linestyle="dashed", label="TTR range in Moby Dick")
    ax.legend()
fig.suptitle("TTR in first half of Holy Grail vs. TTR range in Moby Dick")

# Porovnání se zdrojovým textem

Když se podíváme na zdrojový kód modulu `nltk.book`, snadno zjistíme, odkud se texty, s nimiž jsme pracovali, načítají:

In [ ]:
import nltk

nltk.book??

Jak patrno, *Moby Dick* pochází z korpusu `gutenberg`, *Monty Python* z korpusu `webtext` (tedy aspoň v rámci NLTK).

In [ ]:
from nltk.corpus import webtext, gutenberg

Při konstrukci objektů typu `Text` se z těchto korpusů načítají metodou `words`, protože je chceme mít tokenizované. Ale pro pročítání úryvků se spíš hodí načíst je pomocí metody `raw` jako jeden dlouhý řetězec, kde jsou např. zachované konce řádků a tím i odstavce, což usnadní čtení a orientaci.

In [ ]:
moby = gutenberg.raw("melville-moby_dick.txt")
grail = webtext.raw("grail.txt")

Podle grafu vývoje TTR je scéna 5 v *Monty Python and the Holy Grail* zkraje nápadně lexikálně chudá, a ke konci naopak vystoupá až skoro k maximu v rámci textu. Pojďme se na ni podívat, abychom si ověřili, zda to intuitivně dává smysl:

In [ ]:
print(grail[grail.index("SCENE 5"):grail.index("SCENE 6")])

Vypadá to, že to sedí: první část scény vyvozuje komický efekt z primitivních repetitivních replik, kdežto ke konci se rozvine komplexnější dialog, a scéna je zakončena květnatým vstupem vypravěče.

# Douška: univerzálnější funkce `ttr`

Co když chceme při výpočtu TTR např. zanedbat velikost písmen, nebo provést nějakou jinou úpravu zdrojového textu? Nejsnazší řešení by mělo být prostě jen zaměnit původní kolekci za konvertor kolekce. Jenže ouha:

In [ ]:
sent3

In [ ]:
ttr(sent3)

In [ ]:
ttr(t.lower() for t in sent3)

Výsledkem konvertoru kolekce je totiž objekt, kterému Python říká *generator*:

In [ ]:
(t.lower() for t in sent3)

My o něm na hodinách mluvíváme jako o *potenciální kolekci*. Potenciální (nebo též líná) kolekce sice ví, jak svoje prvky spočítat, ale nechává to na jindy. Je koneckonců líná, takže je potřeba ji donutit -- třeba tak, že z jejích prvků vytvoříme seznam pomocí funkce `list`. Seznam svoje prvky znát potřebuje, a tak generátoru nezbývá než si dát tu práci a spočítat je:

In [ ]:
list(t.lower() for t in sent3)

A protože je generátor líný a nezná svoje prvky (jen má uložený recept, jak je spočítat), tak ani dopředu neví, kolik jich celkem bude, a nelze na něj jen tak zavolat funkci `len`.

In [ ]:
len(t.lower() for t in sent3)

Všimněte si, že je to tatáž chyba, jakou nám před chvílí vyhodila funkce `ttr`, a příčina je pochopitelně taky stejná.

Řešení je jednoduché: stačí zajistit, aby v proměnné `text` byla ve chvíli, kdy voláme `len(text)`, uložena skutečná, nikoli pouze potenciální kolekce (generátor). Pak by mělo vždy jít stanovit délku textu. A jak už jsme si ukázali před chvílí, převést libovolnou kolekci (i potenciální) např. na seznam je naštěstí snadné, stačí použít funkci `list`. Takže v úhrnu:

In [ ]:
def ttr(text):
    text = list(text)  # tenhle řádek je nový
    return len(set(text)) / len(text)

Takto upravené funkci `ttr` už bez problémů můžeme předat jako argument konvertor kolekce, tedy generátor, protože ho hned zkraje funkce převedeme na seznam.

In [ ]:
ttr(t.lower() for t in sent3)